# Dataser Espectroscopia Fotoacustica Simulado para EDA e ML
## Sinais simulados de gases para detecção e caracterização
#### Joel Cassa Neto
#### 31/05/2025

## Gerador de dados
##### 1. Importações

In [4]:
import numpy as np
import pandas as pd
import os

##### 2. Constantes do projeto
Definições das constantes

In [5]:
# Célula 2 - CONSTANTES DO PROJETO

GASES_INFO_PROJETO = {
    'CO2': {
        'formula': 'CO2',
        'nome': 'Dióxido de Carbono',
        'picos_absorcao': [
            {'centro_nm': 2000, 'largura_nm': 50, 'intensidade_max': 0.8},
            {'centro_nm': 2700, 'largura_nm': 60, 'intensidade_max': 0.9},
            {'centro_nm': 4260, 'largura_nm': 70, 'intensidade_max': 1.0}
        ],
        'concentracao_tipica_ppm': 400,
        'faixa_concentracao_ppm': (380, 600) # Faixa para simular variação em ambientes comuns
    },
    'CH4': {
        'formula': 'CH4',
        'nome': 'Metano',
        'picos_absorcao': [
            {'centro_nm': 1650, 'largura_nm': 40, 'intensidade_max': 0.7},
            {'centro_nm': 2300, 'largura_nm': 50, 'intensidade_max': 0.85},
            {'centro_nm': 3300, 'largura_nm': 60, 'intensidade_max': 0.95}
        ],
        'concentracao_tipica_ppm': 2,
        'faixa_concentracao_ppm': (1, 10) # Faixa para simular vazamentos ou concentrações elevadas
    },
    'H2O': {
        'formula': 'H2O',
        'nome': 'Vapor d\'água',
        'picos_absorcao': [
            {'centro_nm': 1350, 'largura_nm': 100, 'intensidade_max': 0.6},
            {'centro_nm': 1850, 'largura_nm': 100, 'intensidade_max': 0.7},
            {'centro_nm': 2600, 'largura_nm': 150, 'intensidade_max': 0.5}
        ],
        'concentracao_tipica_ppm': 10000, # Pode variar muito
        'faixa_concentracao_ppm': (5000, 25000) # Variação grande para umidade do ar
    },
    'CO': {
        'formula': 'CO',
        'nome': 'Monóxido de Carbono',
        'picos_absorcao': [
            {'centro_nm': 2330, 'largura_nm': 30, 'intensidade_max': 0.6},
            {'centro_nm': 4670, 'largura_nm': 50, 'intensidade_max': 0.9}
        ],
        'concentracao_tipica_ppm': 10, # Limites de exposição são baixos, mas para detecção pode variar.
        'faixa_concentracao_ppm': (0.5, 50) # Faixa de detecção para poluição
    },
    'N2O': {
        'formula': 'N2O',
        'nome': 'Óxido Nitroso',
        'picos_absorcao': [
            {'centro_nm': 3900, 'largura_nm': 60, 'intensidade_max': 0.75},
            {'centro_nm': 4500, 'largura_nm': 70, 'intensidade_max': 0.85}
        ],
        'concentracao_tipica_ppm': 0.3, # Gás de efeito estufa com baixa concentração ambiente
        'faixa_concentracao_ppm': (0.2, 1) # Faixa de interesse para monitoramento ambiental
    }
}

COMPRIMENTOS_ONDA_VARREDURA = np.linspace(1000, 5000, 400) # Exemplo: 400 pontos de 1000nm a 5000nm

##### 3. Função para gerar sinal fotoacústico individual
- gerar_sinal_fotoacustico

In [6]:
# Célula 3 - FUNÇÃO PARA CALCULAR O PERFIL DE ABSORÇÃO DE UM GÁS INDIVIDUAL
def calcular_absorcao_para_gas(info_gas, comprimentos_onda):
    """
    Calcula o perfil de absorção (espectro) de um gás específico com base nos seus picos.
    Não inclui concentração nem ruído nesta etapa.
    """
    sinal_base = np.zeros_like(comprimentos_onda, dtype=float)

    for pico in info_gas['picos_absorcao']:
        centro = pico['centro_nm']
        largura = pico['largura_nm']
        intensidade_max = pico['intensidade_max']
        # Modelo Gaussiano para o pico de absorção
        # Usamos largura / 2.355 para converter FWHM para sigma em uma gaussiana
        sigma = largura / 2.355
        componente_pico = intensidade_max * np.exp(-((comprimentos_onda - centro)**2) / (2 * sigma**2))
        sinal_base += componente_pico
        
    return sinal_base

##### 4. Função para gerar Dataset
- gerar_dataset_fotoacustico

In [7]:
# Célula 4 - FUNÇÃO PARA GERAR DATASET DE ESPECTROS FOTOACÚSTICOS DE MISTURAS
# SUBSTITUA TODO O CÓDIGO DESTA CÉLULA PELO SEGUINTE:

def gerar_dataset_misturas_fotoacusticas(num_amostras_por_gas_principal, gases_info, comprimentos_onda, ruido_nivel=0.01):
    """
    Gera um dataset de espectros fotoacústicos simulados para misturas de gases.
    Cada amostra terá concentrações variadas de todos os gases definidos.
    A geração é balanceada para garantir que cada gás tenha a chance de ser o "principal".

    Args:
        num_amostras_por_gas_principal (int): Número de amostras a gerar para cada gás,
                                              onde esse gás será o "principal" (dominante).
        gases_info (dict): Dicionário com informações de cada gás (nome, picos_absorcao,
                           concentracao_tipica_ppm, faixa_concentracao_ppm).
        comprimentos_onda (np.array): Array de comprimentos de onda de varredura (nm).
        ruido_nivel (float): Nível de ruído gaussiano a ser adicionado ao sinal final.

    Returns:
        pd.DataFrame: DataFrame com as colunas de concentrações de cada gás,
                      o gás principal da amostra e os espectros de absorção.
    """
    
    colunas_espectro = [f'lambda_{int(l)}nm' for l in comprimentos_onda]
    df_list = []
    
    # Dicionário para armazenar os espectros de absorção base de cada gás para otimização
    espectros_base_cache = {
        gas_nome: calcular_absorcao_para_gas(info, comprimentos_onda)
        for gas_nome, info in gases_info.items()
    }
    
    gases_disponiveis = list(gases_info.keys())

    # Itera sobre cada gás para garantir que ele seja o "principal" em um número específico de amostras
    for gas_foco_principal in gases_disponiveis:
        info_gas_foco = gases_info[gas_foco_principal]
        min_c_foco, max_c_foco = info_gas_foco['faixa_concentracao_ppm']

        for _ in range(num_amostras_por_gas_principal):
            concentracoes_amostra = {}
            espectro_total_mistura = np.zeros_like(comprimentos_onda, dtype=float)
            
            # --- Gerar concentração para o GÁS PRINCIPAL desta amostra ---
            # Para gases com faixas de concentração naturalmente baixas (como CH4, CO, N2O),
            # garantimos que sua concentração seja alta o suficiente para ser o principal.
            if gas_foco_principal in ['CH4', 'CO', 'N2O']:
                # Amostra em uma faixa que tende a ser maior que a dos gases de fundo
                # Ajuste estes multiplicadores conforme a necessidade de garantir o foco
                conc_foco = np.random.uniform(max_c_foco * 0.8, max_c_foco * 2.0)
            else:
                # Para gases com faixas mais amplas (CO2, H2O), usa a faixa normal
                conc_foco = np.random.uniform(min_c_foco, max_c_foco)
            
            concentracoes_amostra[gas_foco_principal] = conc_foco

            # --- Gerar concentrações para os OUTROS gases (gases de fundo) ---
            for gas_nome_fundo, info_fundo in gases_info.items():
                if gas_nome_fundo != gas_foco_principal:
                    min_c_fundo, max_c_fundo = info_fundo['faixa_concentracao_ppm']
                    
                    # Define uma faixa de concentração menor para os gases de fundo
                    # para que o gás foco seja de fato o principal.
                    # Ajuste esses multiplicadores de fundo conforme a necessidade de mascaramento.
                    conc_fundo = np.random.uniform(min_c_fundo * 0.1, max_c_fundo * 0.5)
                    
                    # Garante que a concentração de fundo não seja zero ou negativa
                    if conc_fundo < 0.001: conc_fundo = 0.001 
                    
                    # --- Tratamento especial para H2O quando NÃO é o gás principal ---
                    # H2O pode ter uma concentração de fundo mais alta mesmo quando não é o foco (umidade ambiente)
                    if gas_nome_fundo == 'H2O':
                        # Mantém uma base de umidade ambiente, mas garante que não ofusque o gás foco
                        conc_h2o_base = np.random.uniform(info_fundo['faixa_concentracao_ppm'][0] * 0.1, info_fundo['faixa_concentracao_ppm'][0] * 0.5)
                        conc_fundo = min(conc_h2o_base, conc_foco * 0.9) # Garante que H2O não seja maior que o gás foco
                        if conc_fundo < 0.001: conc_fundo = 0.001 # Previne concentrações nulas ou negativas
                        
                    concentracoes_amostra[gas_nome_fundo] = conc_fundo
            
            # --- Calcular o espectro total da mistura ---
            # Soma as contribuições de absorção de cada gás, ponderada por sua concentração
            for gas_nome_final, conc_final in concentracoes_amostra.items():
                info_gas_final = gases_info[gas_nome_final]
                espectro_total_mistura += espectros_base_cache[gas_nome_final] * conc_final
            
            # --- Adicionar ruído ao espectro final da mistura ---
            max_sinal_mistura = np.max(espectro_total_mistura)
            if max_sinal_mistura > 0:
                ruido = np.random.normal(0, ruido_nivel * max_sinal_mistura, len(comprimentos_onda))
            else: # Caso a mistura tenha sinal muito baixo
                ruido = np.random.normal(0, ruido_nivel, len(comprimentos_onda)) 

            espectro_final_amostra = espectro_total_mistura + ruido
            espectro_final_amostra[espectro_final_amostra < 0] = 0 # Sinal fotoacústico não pode ser negativo
            
            # --- Determinar o 'gás principal' REAL para esta amostra (para verificação) ---
            # Isso é importante para confirmar que a lógica de "foco" funcionou
            gas_principal_real = max(concentracoes_amostra, key=concentracoes_amostra.get)
            
            # --- Preparar dados para o DataFrame ---
            amostra_data = {
                'gas_principal': gas_principal_real, # O gás que de fato tem a maior concentração
                'concentracao_principal_ppm': concentracoes_amostra[gas_principal_real]
            }
            
            # Adicionar as concentrações de TODOS os gases na amostra como colunas separadas
            for g in gases_disponiveis: # Garante que todas as colunas de concentração existam e na ordem
                amostra_data[f'concentracao_{g}_ppm'] = concentracoes_amostra.get(g, 0.0)

            # Adicionar os dados do espectro
            for j, col_lambda_val in enumerate(comprimentos_onda):
                amostra_data[f'lambda_{int(col_lambda_val)}nm'] = espectro_final_amostra[j]
                
            df_list.append(amostra_data)

    dataset = pd.DataFrame(df_list)
    return dataset

##### 5. Geração e Salvamento do Dataset

In [8]:
# Célula 5 - Parâmetros para a Geração do Dataset e Chamada da Função

# Parâmetros para a Geração do Dataset
num_amostras_por_gas_principal_simulacao = 200 # <-- NOVO PARÂMETRO: Número de amostras para cada gás como principal
ruido_nivel_simulacao = 0.005 # Reduzi um pouco o ruído para um sinal mais limpo, ajuste conforme necessário
nome_arquivo_saida = 'dataset_fotoacustico_misturas_balanceado.csv' # <-- NOVO NOME DE ARQUIVO

# Caminhos para Salvar o Dataset
output_dir = os.path.join('..', 'data')
output_csv_path = os.path.join(output_dir, nome_arquivo_saida)

os.makedirs(output_dir, exist_ok=True)

print(f"Iniciando a geração do dataset com {len(GASES_INFO_PROJETO) * num_amostras_por_gas_principal_simulacao} amostras (sendo {num_amostras_por_gas_principal_simulacao} por gás principal)...")

# Chamada da NOVA função para gerar o dataset completo de misturas
df_simulado = gerar_dataset_misturas_fotoacusticas(
    num_amostras_por_gas_principal=num_amostras_por_gas_principal_simulacao, # <-- Usar o novo parâmetro
    ruido_nivel=ruido_nivel_simulacao,
    gases_info=GASES_INFO_PROJETO,
    comprimentos_onda=COMPRIMENTOS_ONDA_VARREDURA
)

# Salvar o DataFrame gerado em um arquivo CSV
df_simulado.to_csv(output_csv_path, index=False)

print(f"Dataset gerado com {len(df_simulado)} amostras no total.")
print(f"Dataset salvo com sucesso em: {output_csv_path}")
print("\nPrimeiras linhas do dataset gerado:")
print(df_simulado.head())

print("\nContagem de amostras por 'gás principal':")
print(df_simulado['gas_principal'].value_counts())

Iniciando a geração do dataset com 1000 amostras (sendo 200 por gás principal)...
Dataset gerado com 1000 amostras no total.
Dataset salvo com sucesso em: ../data/dataset_fotoacustico_misturas_balanceado.csv

Primeiras linhas do dataset gerado:
  gas_principal  concentracao_principal_ppm  concentracao_CO2_ppm  \
0           CO2                  510.668358            510.668358   
1           CO2                  596.998349            596.998349   
2           CO2                  535.024489            535.024489   
3           CO2                  481.868196            481.868196   
4           CO2                  471.961988            471.961988   

   concentracao_CH4_ppm  concentracao_H2O_ppm  concentracao_CO_ppm  \
0              0.191677            459.601522            16.836195   
1              4.822418            537.298514            23.205567   
2              4.398204            481.522040             1.695562   
3              2.251544            433.681376             9.